<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/funka_alibi1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
import shutil
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, Dense, Layer, Reshape, InputLayer, Flatten, Input, MaxPooling2D




In [ ]:
!git clone https://github.com/SeldonIO/alibi-detect.git





Cloning into 'alibi-detect'...
remote: Enumerating objects: 4042, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4042 (delta 0), reused 0 (delta 0), pack-reused 4038
Receiving objects: 100% (4042/4042), 26.10 MiB | 30.20 MiB/s, done.
Resolving deltas: 100% (2584/2584), done.


In [ ]:
%cd /content/alibi-detect/alibi_detect/od

/content/alibi-detect/alibi_detect/od


In [ ]:
!pip install alibi-detect

In [ ]:
from alibi_detect.od import OutlierAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
def img_to_np(path, resize = True):  
    img_array = []
    fpaths = glob.glob(path, recursive=True)
    for fname in fpaths:
        img = Image.open(fname).convert("RGB")
        if(resize): img = img.resize((64,64))
        img_array.append(np.asarray(img))
    images = np.array(img_array)
    return images
  
path_train = "D:\\img\\capsule\\train\\**\*.*"
path_test = "D:\\img\\capsule\\test\\**\*.*"

In [ ]:
train = img_to_np(path_train)
test = img_to_np(path_test)
train = train.astype('float32') / 255.
test = test.astype('float32') / 255.

In [ ]:
encoding_dim = 1024
dense_dim = [8, 8, 128]

In [ ]:
encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=train[0].shape),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ])

IndexError: ignored

In [ ]:
decoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(encoding_dim,)),
      Dense(np.prod(dense_dim)),
      Reshape(target_shape=dense_dim),
      Conv2DTranspose(256, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
  ])

In [ ]:
od = OutlierAE( threshold = 0.001,
                encoder_net=encoder_net,
                decoder_net=decoder_net)

In [ ]:
adam = tf.keras.optimizers.Adam(lr=1e-4)

In [ ]:
od.fit(train, epochs=100, verbose=True,
       optimizer = adam)

In [ ]:
od.infer_threshold(test, threshold_perc=95)

In [ ]:
preds = od.predict(test, outlier_type='instance',
            return_instance_score=True,
            return_feature_score=True)

In [ ]:
for i, fpath in enumerate(glob.glob(path_test)):
    if(preds['data']['is_outlier'][i] == 1):
        source = fpath
        shutil.copy(source, 'img\\')
        
filenames = [os.path.basename(x) for x in glob.glob(path_test, recursive=True)]

In [ ]:
dict1 = {'Filename': filenames,
     'instance_score': preds['data']['instance_score'],
     'is_outlier': preds['data']['is_outlier']}
     
df = pd.DataFrame(dict1)
df_outliers = df[df['is_outlier'] == 1]

In [ ]:
print(df_outliers)

In [ ]:
recon = od.ae(test).numpy()

In [ ]:
plot_feature_outlier_image(preds, test, 
                           X_recon=recon,  
                           max_instances=5,
                           outliers_only=False,
                           figsize=(15,15))